In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("data/TAQ_15Min_AAPL_2023.csv")
df.index = df.datetime
N_day = 26
df.info()

In [ ]:
def predict_volumes(mu, Sigma, true_volumes):
    """
    param mu: means
    param Sigma: covariance
    param true_volumes: true volumes, i.e., means[0] predicts true_volumes[0]
    """
    n= mu.shape[0]
    
    predictions = np.zeros(n)
    predictions[0] = mu[0]
    
    for i in range(1,n):
        ### following Wikipedia's notation, we observe x2 and update x1
        
        mu1 = mu[i:]
        mu2 = mu[:i]
        a = true_volumes[:i]
        
        mu_bar = mu[i:]
        Sigma_12 = Sigma[i:, :i]
        Sigma_22 = Sigma[:i, :i]
        Sigma_21 = Sigma[:i, i:]
#         assert Sigma_21 == Sigma_12.T

        mu_bar = mu1 + Sigma_12 @ np.linalg.inv(Sigma_22) @ (a-mu2)
        
        predictions[i] = mu_bar[0]
    
    return predictions
    


In [ ]:
X1 = pd.DataFrame(df["VOLUME"])
X1.index = pd.to_datetime(X1.index)
grouped = X1.groupby(pd.Grouper(freq='1d'))

bad_ts = []
to_remove = [pd.Timestamp("2023-07-03"), pd.Timestamp("2023-11-24"), pd.Timestamp("2023-12-24"), pd.Timestamp("2023-12-31")]

for (t,g) in grouped:
    # This removes any days where the exchange closes early, and the days listed in to_remove
    # note that this is necessary because while some of these outlier days have truncated data, others don't
    # so we can't just detect and drop truncated data
    if (g.shape[0] != N_day and g.shape[0] != 0) or (t in to_remove):
        print("Bad timestamp: {} has size {}".format(t, g.shape[0]))
        bad_ts.append(t)

# Delete the days that have truncated data
for bad_t in bad_ts:
    nxt = pd.Timestamp(year=bad_t.year, month=bad_t.month, day=bad_t.day+1)
    X1 = X1[~((X1.index < nxt) & (X1.index > bad_t))]
X1

In [ ]:
N = X1.shape[0]
X = X1.to_numpy().reshape(int(N/N_day), N_day)
X.shape

In [ ]:
def lag_col(pd_col, i):
    d = {"VOLUME" : "VOLUME_{}".format(N_day-i)}
    pd_col.rename(columns=d, inplace=True)
    return pd_col.shift(-i)[:-N_day-1]

#X1 = pd.concat([lag_col(pd.DataFrame(df["VOLUME"]), i) for i in range(N_day)], axis=1)
#X = X1.to_numpy() # this constructs a lagged matrix but I don't know why this makes sense

In [ ]:
np.random.seed(0)

A = np.random.normal(size=(N_day,N_day))

Sigma = A@A.T

#X = np.random.multivariate_normal(np.zeros(10),Sigma, 1000)

mu = X.mean(axis=0)

benchmark_error = ((X-mu)**2).mean()
# so we would do N_day-1 updates per row since we have N_day columns 

mu = X.mean(axis=0)
Sigma = np.cov(X[:-1].T)
true_volumes = X[-1]

pred = predict_volumes(mu, Sigma, true_volumes)
print(f"MSE without mean update: {((mu-true_volumes)**2).mean()}")
print(f"MSE with mean update: {((pred-true_volumes)**2).mean()}")

In [ ]:
plt.semilogy(mu)

In [ ]:
plt.semilogy(range(N_day), pred, label="Predicted", linestyle="dashed")
plt.semilogy(range(-N_day,1), X1["VOLUME"][-N_day*2:-N_day+1], label="Last {} of data".format(N_day), color="black")
plt.semilogy(range(N_day), true_volumes, label="True", color="blue")
plt.legend()

## Visualization of data
This isn't any type of analysis, it's just a good plot for our presentation

In [ ]:
#for d in range(X.shape[0]):
#    plt.semilogy(X[d,:], linewidth=0.5, color="cornflowerblue")

#daily_labels = ["9:30", "10:30", "11:30", "12:30", "1:30", "2:30", "3:30"]
#plt.xticks(ticks=range(0, 26, 4), labels=daily_labels)
#plt.ylabel("Volume (# shares)")
#plt.title("Intraday volume for AAPL over 2023-2024")
#plt.savefig("data_visualization.pdf")
#plt.show()